[adaptado de [Programa de cursos integrados Aprendizado de máquina](https://www.coursera.org/specializations/machine-learning-introduction) de [Andrew Ng](https://www.coursera.org/instructor/andrewng)  ([Stanford University](http://online.stanford.edu/), [DeepLearning.AI](https://www.deeplearning.ai/) ) ]

# Laboratório Prático: Redes Neurais para Reconhecimento de Dígitos Manuscritos, Binário

Neste exercício, você usará uma rede neural para reconhecer os dígitos zero e um escritos à mão.


# Tópicos
- [ 1 - Pacotes ](#1)
- [ 2 - Redes Neurais](#2)
  - [ 2.1 Definição do Problema](#2.1)
  - [ 2.2 Conjunto de Dados](#2.2)
  - [ 2.3 Representação de Modelo](#2.3)
  - [ 2.4 Implementação do Modelo no Tensorflow](#2.4)
    - [ Exercício 1](#ex01)
  - [ 2.5 Implementação do Modelo no NumPy(Propagação Direta no NumPy)](#2.5)
    - [ Exercício 2](#ex02)
  - [ 2.6 Implementação do Modelo NumPy - Vetorizado](#2.6)
    - [ Exercício 3](#ex03)
  - [ 2.7 Parabéns!](#2.7)
  - [ 2.8 Tutorial de Broadcasting no NumPy](#2.8)


In [ ]:
# Baixar arquivos adicionais para o laboratório
!wget https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/nn_adv/class_01/11%20-%20Atividade%20avaliativa%20-%20Redes%20Neurais/lab_utils_ml_adv_assig_week_1.zip
!unzip -n -q lab_utils_ml_adv_assig_week_1.zip

<a name="1"></a>
## 1 - Pacotes 

Primeiro, vamos executar a célula abaixo para importar todos os pacotes de que você precisará durante esta tarefa.
- [numpy](https://numpy.org/) é o pacote fundamental para a computação científica com Python.
- [matplotlib](http://matplotlib.org) é uma biblioteca popular para plotar gráficos em Python.
- [tensorflow](https://www.tensorflow.org/) uma plataforma popular para aprendizado de máquina.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from autils import *
%matplotlib inline

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

**Tensorflow e Keras**  
O Tensorflow é um pacote de aprendizado de máquina desenvolvido pelo Google. Em 2019, o Google integrou o Keras ao Tensorflow e lançou o Tensorflow 2.0. O Keras é uma estrutura(_framework_) desenvolvida de forma independente por François Chollet que cria uma interface simples e centrada em camadas para o Tensorflow. Este curso usará a interface do Keras.

<a name="2"></a>
## 2 - Redes Neurais

Uma regressão logística pode ser estendida para lidar com limites não lineares usando a regressão polinomial. No entanto, para cenários ainda mais complexos, como o reconhecimento de imagens, é preferível usar redes neurais.

<a name="2.1"></a>
### 2.1 Definição do Problema

Neste exercício, você usará uma rede neural para reconhecer dois dígitos escritos à mão, zero e um. Essa é uma tarefa de classificação binária. O reconhecimento automatizado de dígitos manuscritos tem muitas amplamente utilizado atualmente, desde o reconhecimento de códigos postais em envelopes de correio até o reconhecimento de valores escritos em documentos bancários. Você ampliará essa rede para reconhecer todos os 10 dígitos (0-9) em uma tarefa futura. 

Este exercício mostrará como os métodos que você aprendeu podem ser usados para essa tarefa de classificação.

<a name="2.2"></a>
### 2.2 Conjunto de Dados

Você começará carregando o conjunto de dados para essa tarefa.
- A função `load_data()` mostrada abaixo carrega dados nas variáveis `X` e `y`


- O conjunto de dados contém 1.000 exemplos de treinamento de dígitos manuscritos $^1$, aqui limitados a zero e um.

    - Cada exemplo de treinamento é uma imagem em escala de cinza de 20 pixels x 20 pixels do dígito. 
        - Cada pixel é representado por um número de ponto flutuante que indica a intensidade da escala de cinza naquele local. 
        - A grade de 20 por 20 pixels é "desenrolada"(_unrolled_) em um vetor de 400 dimensões. 
        - Cada exemplo de treinamento torna-se uma única linha em nossa matriz de dados `X`. 
        - Isso nos dá uma matriz 1000 x 400 `X` em que cada linha é um exemplo de treinamento de uma imagem de dígito manuscrito.


$$X = 
\left(\begin{array}{cc} 
--- (x^{(1)}) --- \\
--- (x^{(2)}) --- \\
\vdots \\ 
--- (x^{(m)}) --- 
\end{array}\right)$$ 

- A segunda parte do conjunto de treinamento é um vetor uni-dimensional de 1000 x 1 `y` que contém rótulos para o conjunto de treinamento
    - `y = 0` se a imagem tiver o dígito `0`, `y = 1` se a imagem tiver o dígito `1`.

$^1$<sub> Esse é um subconjunto do conjunto de dados de dígitos manuscritos MNIST de Yann Lecun</sub>



In [ ]:
# Carregar conjunto de dados
X, y = load_data()

<a name="toc_89367_2.2.1"></a>
#### 2.2.1 Visualizar as Variáveis
Vamos nos familiarizar mais com seu conjunto de dados.  
- Um bom ponto de partida é imprimir cada variável e ver o que ela contém.

O código abaixo imprime os elementos das variáveis `X` e `y`.

In [ ]:
print ('O primeiro elemento de X é: ', X[0])

In [ ]:
print ('O primeiro elemento de y é: ', y[0,0])
print ('O último elemento de y é: ', y[-1,0])

<a name="toc_89367_2.2.2"></a>
#### 2.2.2 Verificar as Dimensões de suas Variáveis

Outra maneira de se familiarizar com seus dados é visualizar suas dimensões. Imprima a forma de `X` e `y` e veja quantos exemplos de treinamento você tem em seu conjunto de dados.

In [ ]:
print ('O formato de  X é: ' + str(X.shape))
print ('O formato de y é: ' + str(y.shape))

<a name="toc_89367_2.2.3"></a>
#### 2.2.3 Visualizando os Dados

Você começará visualizando um subconjunto do conjunto de treinamento. 
- Na célula abaixo, o código seleciona aleatoriamente 64 linhas de `X`, mapeia cada linha de volta para uma imagem em escala de cinza de 20 pixels por 20 pixels e exibe as imagens juntas. 
- O rótulo de cada imagem é exibido acima da imagem

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Não é necessário modificar nada nessa célula

m, n = X.shape

fig, axes = plt.subplots(8,8, figsize=(8,8))
fig.tight_layout(pad=0.1)

for i,ax in enumerate(axes.flat):
    # Selecionar índices randômicos
    random_index = np.random.randint(m)
    
    # Selecione as linhas correspondentes aos índices aleatórios e
    # remodelar a imagem
    X_random_reshaped = X[random_index].reshape((20,20)).T
    
    # Exibir a imagem
    ax.imshow(X_random_reshaped, cmap='gray')
    
    # Exibir o rótulo da imagem acima
    ax.set_title(y[random_index,0])
    ax.set_axis_off()

<a name="2.3"></a>
### 2.3 Representação do Modelo

A rede neural que você usará neste trabalho é mostrada na figura abaixo. 
- Ela tem três camadas densas com ativações sigmoides.
    - Lembre-se de que nossas entradas são valores de pixel de imagens de dígitos.
    - Como as imagens são de tamanho $20\times20$, isso nos dá $400$ de entradas
    
<img src="images/C2_W1_Assign1.PNG" width="500" height="400">

          
- Os parâmetros têm dimensões que são dimensionadas para uma rede neural com $25$ unidades na camada 1, $15$ unidades na camada 2 e $1$ unidade de saída na camada 3. 

    - Lembre-se de que as dimensões desses parâmetros são determinadas da seguinte forma:
        - Se a rede tiver $s_{in}$ unidades em uma camada e $s_{out}$ unidades na camada seguinte, então 
            - $W$ terá a dimensão de $s_{in} \times s_{out}$.
            - $b$ será um vetor com $s_{out}$ elementos
  
    - Portanto, as formas de `W` e `b` são 
        - camada1: A forma de `W1` é (400, 25) e a forma de `b1` é (25,)
        - camada2: A forma de `W2` é (25, 15) e a forma de `b2` é: (15,)
        - camada3: A forma de `W3` é (15, 1) e a forma de `b3` é: (1,)
>**Observação:** O vetor de bias `b` pode ser representado como uma matriz 1-D (n,) ou 2-D (1,n). O Tensorflow utiliza uma representação 1-D e este laboratório manterá essa convenção.


<a name="2.4"></a>
### 2.4 Implementação do Modelo no Tensorflow


Os modelos do Tensorflow são criados camada por camada. As dimensões de entrada de uma camada ($s_{in}$ acima) são calculadas para você. Você especifica as *dimensões de saída* de uma camada e isso determina a dimensão de entrada da próxima camada. A dimensão de entrada da primeira camada é derivada do tamanho dos dados de entrada especificados na instrução `model.fit` abaixo. 
>**Nota:** Também é possível adicionar uma camada de entrada que especifique a dimensão de entrada da primeira camada. Por exemplo:  
`tf.keras.Input(shape=(400,)), #especifica a forma de entrada`  
Incluiremos isso aqui para esclarecer o dimensionamento de alguns modelos.

<a name="ex01"></a>
### Exercício 1

Abaixo, usando Keras [Sequential model](https://keras.io/guides/sequential_model/) e [Dense Layer](https://keras.io/api/layers/core_layers/dense/) com uma ativação sigmoide para construa a rede descrita acima.

In [ ]:
model = Sequential(
    [               
        tf.keras.Input(shape=(400,)),    #especifique a dimensão da entrada
        ### INICIE SEU CÓDIGO AQUI ###
    
        ### TERMINE SEU CÓDIGO AQUI ###
    ], name = "my_model" 
)                            


In [ ]:
model.summary()

<details>
  <summary><font size="3" color="darkgreen"><b>Saída Esperada (Click para expandir) </b></font></summary>
A função `model.summary()` exibe um resumo útil do modelo. Como especificamos um tamanho de camada de entrada, a forma das matrizes de peso e bias é determinada e o número total de parâmetros por camada pode ser mostrado. Observe que os nomes das camadas podem variar, pois são gerados automaticamente.
    
```
Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
dense (Dense)                (None, 25)                10025     
_________________________________________________________________
dense_1 (Dense)              (None, 15)                390       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 16        
=================================================================
Total params: 10,431
Trainable params: 10,431
Non-trainable params: 0
_________________________________________________________________
```

<details>
  <summary><font size="3" color="darkgreen"><b>Click para Dicas</b></font></summary>
Conforme descrito na aula:
    
```python
model = Sequential(                      
    [                                   
        tf.keras.Input(shape=(400,)),    # especifique a dimensão da entrada (opcional)
        Dense(25, activation='sigmoid'), 
        Dense(15, activation='sigmoid'), 
        Dense(1,  activation='sigmoid')  
    ], name = "my_model"                                    
)                                       
``` 

In [ ]:
# Teste da unidade

from public_tests import *

test_c1(model)

As contagens de parâmetros mostradas no resumo correspondem ao número de elementos nas matrizes de peso e polarização, conforme mostrado abaixo.

In [ ]:
L1_num_params = 400 * 25 + 25  # W1 parameters  + b1 parameters
L2_num_params = 25 * 15 + 15   # W2 parameters  + b2 parameters
L3_num_params = 15 * 1 + 1     # W3 parameters  + b3 parameters
print("Parâmetros da camada 1 = ", L1_num_params, ", Parâmetros da camada 2 = ", L2_num_params, ",  Parâmetros da camada 3 = ", L3_num_params )

Podemos examinar os detalhes do modelo extraindo primeiro as camadas com `model.layers` e, em seguida, extraindo os pesos com `layerx.get_weights()`, conforme mostrado abaixo.

In [ ]:
[layer1, layer2, layer3] = model.layers

In [ ]:
#### Examine o formato dos pesos
W1,b1 = layer1.get_weights()
W2,b2 = layer2.get_weights()
W3,b3 = layer3.get_weights()
print(f"Formato de shape = {W1.shape}, Formato de b1 = {b1.shape}")
print(f"Formato de shape = {W2.shape}, Formato de b2 = {b2.shape}")
print(f"Formato de shape = {W3.shape}, Formato de b3 = {b3.shape}")

**Saída Esperada**
```
W1 shape = (400, 25), b1 shape = (25,)  
W2 shape = (25, 15), b2 shape = (15,)  
W3 shape = (15, 1), b3 shape = (1,)
```

`xx.get_weights` retorna uma matriz NumPy. Também é possível acessar os pesos diretamente em sua forma de tensor. Observe a forma dos tensores na camada final.

In [ ]:
print(model.layers[2].weights)

O código a seguir definirá uma função de perda e executará a descida de gradiente para ajustar os pesos do modelo aos dados de treinamento. Isso será explicado em mais detalhes na próxima aula.

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    X,y,
    epochs=20
)

Para executar o modelo em um exemplo para fazer uma previsão, use [Keras `predict`](https://www.tensorflow.org/api_docs/python/tf/keras/Model). A entrada para `predict` é uma matriz, portanto, o exemplo único é remodelado para ser bidimensional.

In [ ]:
prediction = model.predict(X[0].reshape(1,400))  # 0
print(f" predição de um zero: {prediction}")
prediction = model.predict(X[500].reshape(1,400))  # 1
print(f" predição de um 1:  {prediction}")

O resultado do modelo é interpretado como uma probabilidade. No primeiro exemplo acima, a entrada é um zero. O modelo prevê que a probabilidade de a entrada ser um é quase zero. 
No segundo exemplo, a entrada é um. O modelo prevê que a probabilidade de o input ser um é quase um.
Como no caso da regressão logística, a probabilidade é comparada a um limite para fazer uma previsão final.

In [ ]:
if prediction >= 0.5:
    yhat = 1
else:
    yhat = 0
print(f"predição após limiar: {yhat}")

Vamos comparar as previsões com os rótulos de uma amostra aleatória de 64 dígitos. Isso demora um pouco para ser executado.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Não é necessário modificar nada nessa célula

m, n = X.shape

fig, axes = plt.subplots(8,8, figsize=(8,8))
fig.tight_layout(pad=0.1,rect=[0, 0.03, 1, 0.92])#[esquerda, inferior, direita, superior]

for i,ax in enumerate(axes.flat):
    # Selecionar índices randômicos
    random_index = np.random.randint(m)
    
    # Selecionar as linhas correspondentes aos índices aleatórios e
    # remodelar a imagem
    X_random_reshaped = X[random_index].reshape((20,20)).T
    
    # Exibir a imagem
    ax.imshow(X_random_reshaped, cmap='gray')
    
    # Predizer utilizando a rede neural
    prediction = model.predict(X[random_index].reshape(1,400))
    if prediction >= 0.5:
        yhat = 1
    else:
        yhat = 0
    
    # Exibir o rótulo abaixo da imagem
    ax.set_title(f"{y[random_index,0]},{yhat}")
    ax.set_axis_off()
fig.suptitle("Rótulo, yhat", fontsize=16)
plt.show()

<a name="2.5"></a>
### 2.5 Implementação do Modelo no NumPy(Propagação Direta no NumPy)
Conforme descrito na aula, é possível criar sua própria camada densa usando o NumPy. Isso pode ser utilizado para criar uma rede neural de várias camadas.

<img src="images/C2_W1_dense2.PNG" width="600" height="450">


<a name="ex02"></a>
### Exercício 2

Abaixo, crie uma sub-rotina de camada densa. O exemplo da aula utilizou um loop for para visitar cada unidade (`j`) na camada e executar o produto escalar dos pesos dessa unidade (`W[:,j]`) e somar a polarização da unidade (`b[j]`) para formar `z`. Uma função de ativação `g(z)` é então aplicada a esse resultado. Esta seção não utilizará algumas das operações de matriz descritas nas aulas opcionais. Elas serão exploradas em uma seção posterior.

In [ ]:
def my_dense(a_in, W, b, g):
    """
    Computes dense layer
    Args:
      a_in (ndarray (n, )) : Data, 1 example 
      W    (ndarray (n,j)) : Weight matrix, n features per unit, j units
      b    (ndarray (j, )) : bias vector, j units  
      g    activation function (e.g. sigmoid, relu..)
    Returns
      a_out (ndarray (j,))  : j units
    """
    units = W.shape[1]
    a_out = np.zeros(units)
    ### INICIE SEU CÓDIGO AQUI ###

    ### TERMINE SEU CÓDIGO AQUI ###
    return(a_out)


In [ ]:
# Verificação rápida
x_tst = 0.1*np.arange(1,3,1).reshape(2,)  # (1 exemplo, 2 recursos)
W_tst = 0.1*np.arange(1,7,1).reshape(2,3) # (2 recursos de entrada, 3 recursos de saída)
b_tst = 0.1*np.arange(1,4,1).reshape(3,)  # (3 recursos)
A_tst = my_dense(x_tst, W_tst, b_tst, sigmoid)
print(A_tst)

**Saída Esperada**
```
[0.54735762 0.57932425 0.61063923]
```

<details>
  <summary><font size="3" color="darkgreen"><b>Click para Dicas</b></font></summary>
Conforme descrito na aula:
    
```python
def my_dense(a_in, W, b, g):
    """
    Calcula a camada densa
    Args:
      a_in (ndarray (n, )) : Dados, 1 exemplo
      W    (ndarray (n,j)) : Matriz de pesos, n recursos por unidade, j unidides
      b    (ndarray (j, )) : Vetor de bias vector, j unidades  
      g    função de ativação (e.g. sigmoid, relu..)
    Returns
      a_out (ndarray (j,))  : j unidades
    """
    units = W.shape[1]
    a_out = np.zeros(units)
    for j in range(units):             
        w =                            # Selecione os pesos para a unidade j. Eles estão na coluna j de W
        z =                            # produto escalar de w e a_in + b
        a_out[j] =                     # aplicar a ativação a z
    return(a_out)
```
   
    
<details>
  <summary><font size="3" color="darkgreen"><b>Click para mais dicas</b></font></summary>

    
```python
def my_dense(a_in, W, b, g):
    """
    Calcula a camada densa
    Args:
      a_in (ndarray (n, )) : Dados, 1 exemplo
      W    (ndarray (n,j)) : Matriz de pesos, n recursos por unidade, j unidides
      b    (ndarray (j, )) : Vetor de bias vector, j unidades  
      g    função de ativação (e.g. sigmoid, relu..)
    Returns
      a_out (ndarray (j,))  : j unidades
    """
    units = W.shape[1]
    a_out = np.zeros(units)
    for j in range(units):             
        w = W[:,j]                     
        z = np.dot(w, a_in) + b[j]     
        a_out[j] = g(z)                
    return(a_out)
``` 

In [ ]:
# Teste da unidade

test_c2(my_dense)

A célula a seguir cria uma rede neural de três camadas utilizando a sub-rotina `my_dense` acima.

In [ ]:
def my_sequential(x, W1, b1, W2, b2, W3, b3):
    a1 = my_dense(x,  W1, b1, sigmoid)
    a2 = my_dense(a1, W2, b2, sigmoid)
    a3 = my_dense(a2, W3, b3, sigmoid)
    return(a3)

Podemos copiar os pesos e as tendências treinados do Tensorflow.

In [ ]:
W1_tmp,b1_tmp = layer1.get_weights()
W2_tmp,b2_tmp = layer2.get_weights()
W3_tmp,b3_tmp = layer3.get_weights()

In [ ]:
# realizar predições
prediction = my_sequential(X[0], W1_tmp, b1_tmp, W2_tmp, b2_tmp, W3_tmp, b3_tmp )
if prediction >= 0.5:
    yhat = 1
else:
    yhat = 0
print( "yhat = ", yhat, " label= ", y[0,0])
prediction = my_sequential(X[500], W1_tmp, b1_tmp, W2_tmp, b2_tmp, W3_tmp, b3_tmp )
if prediction >= 0.5:
    yhat = 1
else:
    yhat = 0
print( "yhat = ", yhat, " label= ", y[500,0])

Execute a célula a seguir para ver as previsões do modelo Numpy e do modelo Tensorflow. Isso leva um momento para ser executado.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Não é necessário modificar nada nessa célula

m, n = X.shape

fig, axes = plt.subplots(8,8, figsize=(8,8))
fig.tight_layout(pad=0.1,rect=[0, 0.03, 1, 0.92])#[esquerda, inferior, direita, superior]

for i,ax in enumerate(axes.flat):
    # Selecione índices aleatórios
    random_index = np.random.randint(m)
    
    # Selecione as linhas correspondentes aos índices aleatórios e
    # remodelar a imagem
    X_random_reshaped = X[random_index].reshape((20,20)).T
    
    # Exibir a imagem
    ax.imshow(X_random_reshaped, cmap='gray')

    # Prever usando a rede neural implementada no Numpy
    my_prediction = my_sequential(X[random_index], W1_tmp, b1_tmp, W2_tmp, b2_tmp, W3_tmp, b3_tmp )
    my_yhat = int(my_prediction >= 0.5)

    # Prever usando a rede neural implementada no Tensorflow
    tf_prediction = model.predict(X[random_index].reshape(1,400))
    tf_yhat = int(tf_prediction >= 0.5)
    
    # Exibir o rótulo acima da imagem
    ax.set_title(f"{y[random_index,0]},{tf_yhat},{my_yhat}")
    ax.set_axis_off() 
fig.suptitle("Rótulo, yhat Tensorflow, yhat Numpy", fontsize=16)
plt.show()

<a name="2.6"></a>
### 2.6 Implementação do Modelo NumPy - Vetorizado
Lembre-se das aulas que descrevem operações de vetores e matrizes que podem ser usadas para acelerar os cálculos.
A seguir, descrevemos uma operação de camada que calcula a saída de todas as unidades em uma camada em um determinado exemplo de entrada:
<img src="images/C2_W1_VectorMatrix.PNG" width="600" height="450">

Podemos demonstrar isso usando os exemplos `X` e os parâmetros `W1`, `b1` acima. Usamos o `np.matmul` para realizar a multiplicação de matrizes. Observe que as dimensões de x e W devem ser compatíveis, conforme mostrado no diagrama acima.

In [ ]:
x = X[0].reshape(-1,1)         # vetor coluna (400,1)
z1 = np.matmul(x.T,W1) + b1    # (1,400)(400,25) = (1,25)
a1 = sigmoid(z1)
print(a1.shape)

Você pode dar um passo adiante e calcular todas as unidades de todos os exemplos em uma operação Matrix-Matrix.

<img src="images/C2_W1_MatrixMatrix.PNG" width="600" height="450">

A operação completa é $\mathbf{Z}=\mathbf{XW}+\mathbf{b}$. Isso utilizará a transmissão do NumPy para expandir $\mathbf{b}$ para $m$ linhas. Se não estiver familiarizado com isso, há um breve tutorial no final do notebook.

<a name="ex03"></a>
### Exercício 3

Abaixo, componha uma nova sub-rotina `my_dense_v` que execute os cálculos de camada para uma matriz de exemplos. Isso utilizará `np.matmul()`.

In [ ]:
def my_dense_v(A_in, W, b, g):
    """
    Calcula a camada densa
    Args:
      A_in (ndarray (m,n)) : Dados, m exemplos, cada um com n recursos  
      W    (ndarray (n,j)) : Weight matrix, n recursos por unidade, j unidades
      b    (ndarray (1,j)) : vetor de bias, j unidades  
      g    função de ativação (e.g. sigmoid, relu..)
    Returns
      A_out (tf.Tensor or ndarray (m,j)) : m exemplos, j unidades
    """
    ### INICIE SEU CÓDIGO AQUI ###

    ### TERMINE SEU CÓDIGO AQUI ###
    return(A_out)

In [ ]:
X_tst = 0.1*np.arange(1,9,1).reshape(4,2) # (4 exemplos, 2 recursos)
W_tst = 0.1*np.arange(1,7,1).reshape(2,3) # (2 recursos de entrada, 3 recursos de saída)
b_tst = 0.1*np.arange(1,4,1).reshape(1,3) # (1,3 recursos)
A_tst = my_dense_v(X_tst, W_tst, b_tst, sigmoid)
print(A_tst)

**Saída Esperada**

```
[[0.54735762 0.57932425 0.61063923]
 [0.57199613 0.61301418 0.65248946]
 [0.5962827  0.64565631 0.6921095 ]
 [0.62010643 0.67699586 0.72908792]]
 ```

<details>
  <summary><font size="3" color="darkgreen"><b>Click para Dicas</b></font></summary>
    Na forma de matriz, isso pode ser escrito em uma ou duas linhas. 
    
       Z = np.matmul de A_in e W mais b    
       A_out é g(Z)

<details>
  <summary><font size="3" color="darkgreen"><b>Clique para código</b></font></summary>

```python
def my_dense_v(A_in, W, b, g):
    """
    Calcula a camada densa
    Args:
      A_in (ndarray (m,n)) : Dados, m exemplos, cada um com n recursos  
      W    (ndarray (n,j)) : Weight matrix, n recursos por unidade, j unidades
      b    (ndarray (1,j)) : vetor de bias, j unidades  
      g    função de ativação (e.g. sigmoid, relu..)
    Returns
      A_out (tf.Tensor or ndarray (m,j)) : m exemplos, j unidades
    """
    Z = np.matmul(A_in,W) + b    
    A_out = g(Z)                 
    return(A_out)
```


In [ ]:
# Teste da unidade

test_c3(my_dense_v)

A célula a seguir cria uma rede neural de três camadas utilizando a sub-rotina `my_dense_v` acima.

In [ ]:
def my_sequential_v(X, W1, b1, W2, b2, W3, b3):
    A1 = my_dense_v(X,  W1, b1, sigmoid)
    A2 = my_dense_v(A1, W2, b2, sigmoid)
    A3 = my_dense_v(A2, W3, b3, sigmoid)
    return(A3)

Podemos copiar novamente os pesos e os vieses treinadas do Tensorflow.

In [ ]:
W1_tmp,b1_tmp = layer1.get_weights()
W2_tmp,b2_tmp = layer2.get_weights()
W3_tmp,b3_tmp = layer3.get_weights()

Vamos fazer uma previsão com o novo modelo. Isso fará uma previsão de *todos os exemplos de uma vez*. Observe a forma do resultado.

In [ ]:
Prediction = my_sequential_v(X, W1_tmp, b1_tmp, W2_tmp, b2_tmp, W3_tmp, b3_tmp )
Prediction.shape

Aplicaremos um limite de 0,5 como antes, mas a todas as previsões de uma só vez.

In [ ]:
Yhat = (Prediction >= 0.5).astype(int)
print("predict a zero: ",Yhat[0], "predict a one: ", Yhat[500])

Execute a célula a seguir para ver as previsões. Isso usará as previsões que acabamos de calcular acima. A execução demora um pouco.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Não é necessário modificar nada nessa célula

m, n = X.shape

fig, axes = plt.subplots(8, 8, figsize=(8, 8))
fig.tight_layout(pad=0.1, rect=[0, 0.03, 1, 0.92]) #[left, bottom, right, top]

for i, ax in enumerate(axes.flat):
    # Selecione índices aleatórios
    random_index = np.random.randint(m)
    
    # Selecione as linhas correspondentes aos índices aleatórios e
    # remodelar a imagem
    X_random_reshaped = X[random_index].reshape((20, 20)).T
    
    # Exibir a imagem
    ax.imshow(X_random_reshaped, cmap='gray')
   
    # Exibir o rótulo acima da imagem
    ax.set_title(f"{y[random_index,0]}, {Yhat[random_index, 0]}")
    ax.set_axis_off() 
fig.suptitle("Label, Yhat", fontsize=16)
plt.show()

Você pode ver a aparência de uma das imagens classificadas incorretamente.

In [ ]:
fig = plt.figure(figsize=(1, 1))
errors = np.where(y != Yhat)
random_index = errors[0][0]
X_random_reshaped = X[random_index].reshape((20, 20)).T
plt.imshow(X_random_reshaped, cmap='gray')
plt.title(f"{y[random_index,0]}, {Yhat[random_index, 0]}")
plt.axis('off')
plt.show()

<a name="2.7"></a>
### 2.7 Parabéns!
Você construiu e utilizou uma rede neural com sucesso.

<a name="2.8"></a>
### 2.8 2.8 Tutorial de Broadcasting no NumPy


No último exemplo, $\mathbf{Z}=\mathbf{XW} + \mathbf{b}$ utilizou o broadcasting  do NumPy para expandir o vetor $\mathbf{b}$. Se não estiver familiarizado com o _NumPy Broadcasting_, este breve tutorial é pra você!

$\mathbf{XW}$ é uma operação de matriz-matriz com dimensões $(m,j_1)(j_1,j_2)$ que resulta em uma matriz com dimensão $(m,j_2)$. A isso, adicionamos um vetor $\mathbf{b}$ com dimensão $(1,j_2)$. $\mathbf{b}$ deve ser expandido para ser uma matriz $(m,j_2)$ para que essa operação de elementos faça sentido. Essa expansão é realizada para você pelo _NumPy Broadcasting_.

A _broadcasting_ se aplica a operações que envolvem operações elemento-a-elemento.  
Sua operação básica é "esticar" uma dimensão menor replicando elementos para corresponder a uma dimensão maior.

Mais [especificamente](https://NumPy.org/doc/stable/user/basics.broadcasting.html): 
Ao operar em duas matrizes, o NumPy compara suas formas em termos de elementos. Ele começa com as dimensões mais à direita e segue para a esquerda. Duas dimensões são compatíveis quando
- são iguais, ou
- uma delas é 1   

Se essas condições não forem atendidas, será lançada uma exceção _ValueError: operands could not be broadcast together_, indicando que as matrizes têm formas incompatíveis. O tamanho da matriz resultante é o tamanho que não é 1 ao longo de cada eixo das entradas.

Aqui estão alguns exemplos:

<figure>
    <center> <img src="./images/C2_W1_Assign1_BroadcastIndexes.PNG"  alt='missing' width="400"  ><center/>
    <figcaption>Calculando o formato resultante do Broadcast</figcaption>
<figure/>

O gráfico abaixo descreve as dimensões de expansão. Observe o texto em vermelho abaixo:

<figure>
    <center> <img src="./images/C2_W1_Assign1_Broadcasting.gif"  alt='missing' width="600"  ><center/>
    <figcaption>O Broadcast expande os argumentos para que correspondam a operações com elementos</figcaption>
<figure/>

O gráfico acima mostra o NumPy expandindo os argumentos para que combinem antes da operação final. Observe que essa é uma descrição fictícia. A mecânica real da operação do NumPy escolhe a implementação mais eficiente.

Para cada um dos exemplos a seguir, tente adivinhar o tamanho do resultado antes de executar o exemplo.

In [ ]:
a = np.array([1,2,3]).reshape(-1,1)  #(3,1)
b = 5
print(f"(a + b).shape: {(a + b).shape}, \na + b = \n{a + b}")

Observe que isso se aplica a todas as operações com elemento-a-elemento:

In [ ]:
a = np.array([1,2,3]).reshape(-1,1)  #(3,1)
b = 5
print(f"(a * b).shape: {(a * b).shape}, \na * b = \n{a * b}")

<figure>
    <img src="./images/C2_W1_Assign1_VectorAdd.PNG"  alt='missing' width="740" >
    <center><figcaption><b>Row-Column Element-Wise Operations</b></figcaption></center>
<figure/>

In [ ]:
a = np.array([1,2,3,4]).reshape(-1,1)
b = np.array([1,2,3]).reshape(1,-1)
print(a)
print(b)
print(f"(a + b).shape: {(a + b).shape}, \na + b = \n{a + b}")

Esse é o cenário na camada densa que você criou acima. Adição de um vetor 1-D $b$ a uma matriz (m,j).
<figure>
    <img src="./images/C2_W1_Assign1_BroadcastMatrix.PNG"  alt='missing' width="740" >
    <center><figcaption><b>Matrix + Vetor 1-D</b></figcaption></center>
<figure/>